In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import itertools

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.applications.resnet50 import ResNet50

In [23]:
train_dir = r"C:\Users\Aziz Shaikh\Desktop\mywork\skin-disease-build\dataset\train"
test_dir = r"C:\Users\Aziz Shaikh\Desktop\mywork\skin-disease-build\dataset\test"

In [24]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,   # randomly flip images
        validation_split = 0.2) 

In [25]:
train_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'training')
valid_gen = datagen.flow_from_directory(train_dir,
                                       batch_size=32,
                                       class_mode='sparse',
                                       target_size=(200,200),
                                       subset = 'validation')

Found 12453 images belonging to 23 classes.
Found 3104 images belonging to 23 classes.


In [26]:
base_model = ResNet50(include_top=False, input_shape=(200, 200, 3),pooling = 'avg')
model = Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(23, activation = 'softmax'))

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 4d473c1dd8becc155b73f8504c6f6626 so we will re-download the data.
94773248/94765736 [==============================] - 28s 0us/step


In [27]:
for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-25:]:
    layer.trainable = True

In [28]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer = optimizer , loss = "sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])

In [29]:
epochs = 50

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 23)                2967      
Total params: 23,852,951
Trainable params: 9,204,759
Non-trainable params: 14,648,192
_________________________________________________________________


In [ ]:
history = model.fit_generator(train_gen,epochs = epochs,
                             validation_data = valid_gen,
                             steps_per_epoch = train_gen.n//train_gen.batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
 73/389 [====>.........................] - ETA: 31:47 - loss: 3.0156 - sparse_categorical_accuracy: 0.1396